In [1]:
# Import necessary libraries

import numpy as np
import os
import pandas as pd

In [2]:
# Get verified labels ID

os.chdir("/home/jantina/CoralNet/Image_Metadata")
data = pd.read_excel('verified_labels.xlsx',sheet_name='Health_Labels',usecols="A,C,D").dropna()
data["Label ID"] = data["Label ID"].astype(int)

In [3]:
# Extract data from all CoralNet sources

# Assign directory
directory = '/data/jantina/CoralNet'

# Declare variables
names = []
images = []
labels = []
annots = []
health_labels = []
average_health_labels = []
 
# Iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    os.chdir(f)
    
    try:
        # Get the image name
        names.append(f.split("/")[4])
        
        # Get number of images
        landmarks_frame = pd.read_csv('annotations.csv')
        temp = landmarks_frame.groupby('Name').count()
        images.append(len(temp))
        
        # Get number of labels 
        label = pd.read_csv('labelset.csv')
        labels.append(len(label))
        
        # Get number of labels per image
        annots.append(temp["Label"][0])
        
        # Get number of health labels 
        label["Label ID"] = label["Label ID"].astype(int)
        df_common = label.loc[label["Label ID"].isin(data["Label ID"])]
        temp2 = landmarks_frame.groupby('Label').count()
        df_common = temp2.loc[temp2.index.isin(df_common["Short Code"])]
        health_labels.append(len(df_common))
        
        # Get the average of health labels/images
        average_health_labels.append(df_common["Name"].mean())
        
    except:
        continue

In [4]:
# Create dataframe with all information

df = pd.DataFrame(list(zip(names,
                           images,
                           labels,
                           annots, 
                           health_labels,
                           average_health_labels)),
               columns = ['Name', 
                         '# Images',
                         '# Labels', 
                         '# Labels/Images',
                         '# Health Labels', 
                         'Average # Health Labels/Images'])

df.sort_values('Name', inplace = True,)

In [5]:
# Save metadata as excel file

os.chdir("/home/jantina/CoralNet/Image_Metadata")
df.to_excel("sources_metadata.xlsx")